In [ ]:
from imutils.perspective import four_point_transform
from imutils import contours
import imutils
import cv2
import numpy as np

In [ ]:
# load the example image
image = cv2.imread("test.jpg")
im = cv2.imread("test.jpg")
 
# pre-process the image by resizing it, converting it to
# graycale, blurring it, and computing an edge map
image = imutils.resize(image, height=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 50, 200, 255)

In [ ]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
displayCnt = None
 
# loop over the contours
for c in cnts:
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
	# if the contour has four vertices, then we have found
	# the thermostat display
	if len(approx) == 4:
		displayCnt = approx
		break

In [ ]:
cv2.imwrite( "warped.jpg", warped)
cv2.imwrite( "output.jpg", output )

In [ ]:
warped = four_point_transform(gray, displayCnt.reshape(4, 2))
output = four_point_transform(image, displayCnt.reshape(4, 2))

thresh = cv2.adaptiveThreshold(warped,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,21,5)

_ , contours,hierarchy = cv2.findContours(thresh,cv2.RETR_CCOMP ,cv2.CHAIN_APPROX_SIMPLE)

samples =  np.empty((0,100))
responses = []
keys = [i for i in range(48,58)]

print(len(contours))

digitCnts = []

for cnt in contours:
    if cv2.contourArea(cnt)>5:
        cntRec = cv2.boundingRect(cnt)
        [x,y,w,h] = cntRec
        if ((w >= 10 and w <= 100) and (h >= 20 and h <= 200)) or ((w >= 5 and w <= 20) and (h >= 5 and h <= 20) and (abs(w-h) < 2)):
            digitCnts.append(cntRec)
            
print(len(digitCnts))
  
 
for cntRec in digitCnts:
    [x,y,w,h] = cntRec
    cv2.rectangle(thresh,(x,y),(x+w,y+h),(0,0,255),2)
    roi = thresh[y:y+h,x:x+w]
    roismall = cv2.resize(roi,(10,10))
    cv2.imshow('norm',thresh)
    key = cv2.waitKey(0)
    if key == 27:  # (escape to quit)
        sys.exit()
    elif key in keys:
        responses.append(int(chr(key)))
        sample = roismall.reshape((1,100))
        samples = np.append(samples,sample,0)
                
#cv2.imshow('norm',im2)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

86
18
